# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudness,Wind speed,Country,Date
0,0,albany,42.6001,-73.9662,72.68,91,40,4.38,US,1623289120
1,1,jiwani,25.0500,61.7417,83.07,83,76,6.91,PK,1623289177
2,2,ipixuna,-1.7625,-48.8043,76.95,98,100,1.79,BR,1623289177
3,3,kapaa,22.0752,-159.3190,85.98,68,1,4.00,US,1623289035
4,4,puerto ayora,-0.7393,-90.3518,73.54,91,2,5.01,EC,1623289178
...,...,...,...,...,...,...,...,...,...,...
548,548,daru,-9.0763,143.2092,80.62,81,71,8.48,PG,1623289323
549,549,corpus christi,27.8006,-97.3964,84.22,83,20,4.00,US,1623289073
550,550,leo,11.1000,-2.1000,75.85,79,72,2.62,BF,1623289324
551,551,hokitika,-42.7167,170.9667,57.72,63,96,1.28,NZ,1623289324


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
#print(api_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]
#print(locations)

# Store Humidity in humidity
humidity = weather_data["Humidity"]
#print(humidity)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

max_intensity = np.max(humidity)
#print(max_intensity)

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind speed"] <= 10) & (weather_data["Cloudness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudness,Wind speed,Country,Date
9,9,marrakesh,31.6315,-8.0083,78.87,38,0,4.61,MA,1623289179
139,139,katsuura,35.1333,140.3000,79.23,68,0,8.03,JP,1623289213
162,162,hasaki,35.7333,140.8333,79.29,66,0,7.18,JP,1623289220
271,271,arona,28.0996,-16.6810,75.70,45,0,2.28,ES,1623289248
322,322,vao,-22.6667,167.4833,70.54,66,0,9.69,NC,1623289262
422,422,cherat,33.8233,71.8915,78.73,18,0,2.30,PK,1623289289
437,437,gasa,31.5000,34.4667,73.90,68,0,7.83,PS,1623289293
449,449,rosetta,31.4044,30.4164,71.42,77,0,6.31,EG,1623289296
502,502,chifeng,42.2683,118.9636,77.16,29,0,3.85,CN,1623289222
515,515,shingu,33.7333,135.9833,79.92,62,0,8.08,JP,1623289313


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,marrakesh,MA,31.6315,-8.0083,
139,katsuura,JP,35.1333,140.3000,
162,hasaki,JP,35.7333,140.8333,
271,arona,ES,28.0996,-16.6810,
322,vao,NC,-22.6667,167.4833,
422,cherat,PK,33.8233,71.8915,
437,gasa,PS,31.5000,34.4667,
449,rosetta,EG,31.4044,30.4164,
502,chifeng,CN,42.2683,118.9636,
515,shingu,JP,33.7333,135.9833,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    #locations = weather_data[["Lat", "Lng"]]
    #print(params)
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    #print(response)
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 9: marrakesh.
Closest hotel in marrakesh is Sofitel Marrakech Lounge & Spa.
------------
Retrieving Results for Index 139: katsuura.
Closest hotel in katsuura is Katsuura Hilltop Hotel & Residence.
------------
Retrieving Results for Index 162: hasaki.
Closest hotel in hasaki is Villa Umi To Mori.
------------
Retrieving Results for Index 271: arona.
Closest hotel in arona is Hotel Jardín Tropical.
------------
Retrieving Results for Index 322: vao.
Closest hotel in vao is OURE TERA beach resort.
------------
Retrieving Results for Index 422: cherat.
Missing field/result... skipping.
------------
Retrieving Results for Index 437: gasa.
Missing field/result... skipping.
------------
Retrieving Results for Index 449: rosetta.
Missing field/result... skipping.
------------
Retrieving Results for Index 502: chifeng.
Closest hotel in chifeng is Wanda Realm Chifeng.
------------
Retrieving Results for Index 515: shingu.
Closest hotel in shingu is Hotel New Palace

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,marrakesh,MA,31.6315,-8.0083,Sofitel Marrakech Lounge & Spa
139,katsuura,JP,35.1333,140.3000,Katsuura Hilltop Hotel & Residence
162,hasaki,JP,35.7333,140.8333,Villa Umi To Mori
271,arona,ES,28.0996,-16.6810,Hotel Jardín Tropical
322,vao,NC,-22.6667,167.4833,OURE TERA beach resort
422,cherat,PK,33.8233,71.8915,
437,gasa,PS,31.5000,34.4667,
449,rosetta,EG,31.4044,30.4164,
502,chifeng,CN,42.2683,118.9636,Wanda Realm Chifeng
515,shingu,JP,33.7333,135.9833,Hotel New Palace


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map (fig)
fig

Figure(layout=FigureLayout(height='420px'))